Written by Udi Aharoni  
Based on "The Hitchhiker’s Guide to Altruism: Gene-culture Coevolution, and the Internalization of Norms" Herbert Gintis

Create basic variables:

In [ ]:
from  sympy import *

# system params
u, t, gamma= symbols("u t gamma")

# current generation
p_aac, p_aad, p_abc, p_abd, p_bbd= symbols("p_aac p_aad p_abc p_abd p_bbd")

Fitness, and other simple derived variables:

In [ ]:
p_bbd=(1-p_aac-p_aad-p_abc-p_abd)


f_aac=p_aac*(1-u)*(1+t)
f_abc=p_abc*(1-u)*(1+t)
f_aad=p_aad*(1-u)
f_abd=p_abd*(1-u)
f_bbd=p_bbd

f_average=f_aac + f_aad + f_abc +  f_abd + f_bbd



 Place them in a dictonary for easy access later:

In [ ]:
adultP={
    "aaC": f_aac,
     "abC": f_abc,
     "aaD": f_aad,
     "abD": f_abd,
     "bbD": f_bbd
}

def printDictionary(dict):
  for x in dict:
    print(x,":",dict[x])


printDictionary(adultP)


aaC : p_aac*(t + 1)*(-u + 1)
abC : p_abc*(t + 1)*(-u + 1)
aaD : p_aad*(-u + 1)
abD : p_abd*(-u + 1)
bbD : -p_aac - p_aad - p_abc - p_abd + 1


Construct families:

In [ ]:
families={}

for m in adultP:  
  for f in adultP:

    fam=m+f
    # as in the paper, sort gene-phenotypes lexicographically in family id
    if (f<m):
      fam=f+m

    prob=adultP[m]*adultP[f]
        
    if fam in families:
      families[fam]=families[fam]+prob
    else:
      families[fam]=prob

printDictionary(families)


aaCaaC : p_aac**2*(t + 1)**2*(-u + 1)**2
aaCabC : 2*p_aac*p_abc*(t + 1)**2*(-u + 1)**2
aaCaaD : 2*p_aac*p_aad*(t + 1)*(-u + 1)**2
aaCabD : 2*p_aac*p_abd*(t + 1)*(-u + 1)**2
aaCbbD : 2*p_aac*(t + 1)*(-u + 1)*(-p_aac - p_aad - p_abc - p_abd + 1)
abCabC : p_abc**2*(t + 1)**2*(-u + 1)**2
aaDabC : 2*p_aad*p_abc*(t + 1)*(-u + 1)**2
abCabD : 2*p_abc*p_abd*(t + 1)*(-u + 1)**2
abCbbD : 2*p_abc*(t + 1)*(-u + 1)*(-p_aac - p_aad - p_abc - p_abd + 1)
aaDaaD : p_aad**2*(-u + 1)**2
aaDabD : 2*p_aad*p_abd*(-u + 1)**2
aaDbbD : 2*p_aad*(-u + 1)*(-p_aac - p_aad - p_abc - p_abd + 1)
abDabD : p_abd**2*(-u + 1)**2
abDbbD : 2*p_abd*(-u + 1)*(-p_aac - p_aad - p_abc - p_abd + 1)
bbDbbD : (-p_aac - p_aad - p_abc - p_abd + 1)**2


Normalization and sanity checks:

In [ ]:
# The normalization factor should be f_average squared:
fac=f_average**2

# Normalize
nfamilies={}
for x in families:
  nfamilies[x]=families[x]/fac;

def assertSum1(dict):    
  s=sum(dict.values());  
  s=simplify(s)   
  print("Sum=",s)  
  assert(s==1)  
  print ("Normalization test passed");

assertSum1(nfamilies)

Sum= 1
Normalization test passed


Construct children frequencies:

In [ ]:

p_c=(f_aac+f_abc)/f_average


childP={}

def addChild(c,prob):
  if c in childP:
    childP[c]=childP[c]+prob
  else:
    childP[c]=prob

for fam in nfamilies:
  famP=nfamilies[fam];
  
  # choose first alele
  for a1 in range(2):
    # choose second alele
    for a2 in range(2):

      # get alleles
      aa1=fam[a1]
      aa2=fam[3+a2];
      # sort them 
      if (aa2<aa1):
        aa1,aa2=aa2,aa1        

      # choose phenotype
      ps=[fam[2],fam[5]]
      all_b= aa1=="b" and aa2=="b"
      if all_b:
        ps=["D"]               

      for pp in ps:
        child=aa1+aa2+pp
        prob=famP/4/len(ps)
        if ((not all_b) and pp=="D"):
          probCulture=gamma * p_c
          addChild(aa1+aa2+"C",prob*probCulture)
          addChild(child,prob*(1-probCulture))
        else:
          addChild(child,prob)
  
printDictionary(childP)        
#assertSum1(childP)


aaC : gamma*p_aac*p_aad*(t + 1)*(-u + 1)**2*(p_aac*(t + 1)*(-u + 1) + p_abc*(t + 1)*(-u + 1))/(p_aac*(t + 1)*(-u + 1) - p_aac + p_aad*(-u + 1) - p_aad + p_abc*(t + 1)*(-u + 1) - p_abc + p_abd*(-u + 1) - p_abd + 1)**3 + gamma*p_aac*p_abd*(t + 1)*(-u + 1)**2*(p_aac*(t + 1)*(-u + 1) + p_abc*(t + 1)*(-u + 1))/(2*(p_aac*(t + 1)*(-u + 1) - p_aac + p_aad*(-u + 1) - p_aad + p_abc*(t + 1)*(-u + 1) - p_abc + p_abd*(-u + 1) - p_abd + 1)**3) + gamma*p_aad**2*(-u + 1)**2*(p_aac*(t + 1)*(-u + 1) + p_abc*(t + 1)*(-u + 1))/(p_aac*(t + 1)*(-u + 1) - p_aac + p_aad*(-u + 1) - p_aad + p_abc*(t + 1)*(-u + 1) - p_abc + p_abd*(-u + 1) - p_abd + 1)**3 + gamma*p_aad*p_abc*(t + 1)*(-u + 1)**2*(p_aac*(t + 1)*(-u + 1) + p_abc*(t + 1)*(-u + 1))/(2*(p_aac*(t + 1)*(-u + 1) - p_aac + p_aad*(-u + 1) - p_aad + p_abc*(t + 1)*(-u + 1) - p_abc + p_abd*(-u + 1) - p_abd + 1)**3) + gamma*p_aad*p_abd*(-u + 1)**2*(p_aac*(t + 1)*(-u + 1) + p_abc*(t + 1)*(-u + 1))/(p_aac*(t + 1)*(-u + 1) - p_aac + p_aad*(-u + 1) - p_aad + p_abc*

Build Jacobian matrix

In [ ]:
vars1=[p_aac, p_aad, p_abc, p_abd]
vars2=[childP["aaC"],childP["aaD"],childP["abC"],childP["abD"]]
n=len(vars1)
J=zeros(n,n)
for i in range(n):
  for j in range(n):
    J[i,j]=diff(vars2[i],vars1[j])




Solve for specific cases

In [ ]:
def solveJ(title,distr):
  n=len(distr)
  subList=[(vars1[i],distr[i]) for i in range(n)]
  #print(subList)

  newJ=zeros(n,n)
  for i in range(n):
    for j in range(n):
      newJ[i,j]=J[i,j].subs(subList)

  ev=newJ.eigenvals()
  print("Solution for: ",title)
  for x in ev:
    print("\tEigenval:",x,"\t\t\tmultiplicity:",ev[x])
  #print(ev)
#  print(newJ)

solveJ("pure aaC",[1,0,0,0])
solveJ("pure aaD",[0,1,0,0])
solveJ("pure bbD",[0,0,0,0])

Solution for:  pure aaC
	Eigenval: 1 			multiplicity: 1
	Eigenval: -(gamma - 1)/(t + 1) 			multiplicity: 1
	Eigenval: -(gamma - 1)/(2*t + 2) 			multiplicity: 1
	Eigenval: 0 			multiplicity: 1
Solution for:  pure aaD
	Eigenval: 1 			multiplicity: 1
	Eigenval: t/2 + 1/2 			multiplicity: 1
	Eigenval: gamma*t + gamma + t + 1 			multiplicity: 1
	Eigenval: 0 			multiplicity: 1
Solution for:  pure bbD
	Eigenval: -u + 1 			multiplicity: 1
	Eigenval: -t*u/2 + t/2 - u/2 + 1/2 			multiplicity: 1
	Eigenval: 0 			multiplicity: 2
